# Routing

Routing allows you to define routes between component ports


In [ ]:
import pp
pp.set_plot_options(show_subports=False)

In [ ]:
c = pp.Component()
mmi1 = c << pp.components.mmi1x2()
mmi2 = c << pp.components.mmi1x2()
mmi2.move((100, 50))
c

## Get_route

`get_route` connects using Manhattan routing

In [ ]:
pp.routing.get_route?

In [ ]:
c = pp.Component('sample_connect')
mmi1 = c << pp.components.mmi1x2()
mmi2 = c << pp.components.mmi1x2()
mmi2.move((100, 50))
route = pp.routing.get_route(mmi1.ports["E1"], mmi2.ports["W0"])
c.add(route.references)
c

In [ ]:
route

**Connect strip: Problem**

sometimes there are obstacles that connect strip does not see!

In [ ]:
c = pp.Component('sample_problem')
mmi1 = c << pp.components.mmi1x2()
mmi2 = c << pp.components.mmi1x2()
mmi2.move((110, 50))
x = c << pp.components.cross(length=20)
x.move((135, 20))
route = pp.routing.get_route(mmi1.ports["E1"], mmi2.ports["E1"])
c.add(route.references)
c

**Solution: Connect strip way points**

You can also specify the points along the route

In [ ]:
pp.routing.get_route_waypoints?

In [ ]:
c = pp.Component('sample_avoid_obstacle')
mmi1 = c << pp.components.mmi1x2()
mmi2 = c << pp.components.mmi1x2()
mmi2.move((110, 50))
x = c << pp.components.cross(length=20)
x.move((135, 20))

x0 = mmi1.ports["E0"].x
y0 = mmi1.ports["E0"].y


x2 = mmi2.ports["E0"].x
y2 = mmi2.ports["E0"].y

route = pp.routing.get_route_from_waypoints([(x0, y0), (x2 + 40, y0), (x2 + 40, y2), (x2, y2)])
c.add(route.references)
c

In [ ]:
route.length

In [ ]:
route.ports

In [ ]:
route.references

## Route with custom transition

Lets say that we want to taper to a different waveguide crosssection, for example, a rib waveguide for lower loss.

In [ ]:
c = pp.Component('sample_connect')
mmi1 = c << pp.components.mmi1x2()
mmi2 = c << pp.components.mmi1x2()
mmi2.move((200, 50))
route = pp.routing.get_route(mmi1.ports["E1"], mmi2.ports["W0"])
c.add(route.references)
c

In [ ]:
import pp

xsection = pp.cross_section(width=0.5, layer=(2,0))

c = pp.Component('sample_connect')
mmi1 = c << pp.components.mmi1x2()
mmi2 = c << pp.components.mmi1x2()
mmi2.move((200, 50))
x0 = mmi1.ports["E1"].midpoint[0]
y0 = mmi1.ports["E1"].midpoint[1]
xc = 100
x1 = mmi2.ports["W0"].midpoint[0]
y1 = mmi2.ports["W0"].midpoint[1]

#path = pp.path.smooth((mmi1.ports["E1"].midpoint, (100,40), mmi2.ports["W0"].midpoint))
path = pp.path.smooth([(x0, y0), (xc, y0), (xc, y1), (x1, y1)])
wg = pp.path.extrude(path, xsection)
c<< wg
c

## Routing bundles

**Problem**: when connecting groups of ports there are collisions if we use a single port routing function such as `connect strip`

In [ ]:
import pp

xs_top = [0, 10, 20, 40, 50, 80]
pitch = 127
N = len(xs_top)
xs_bottom = [(i - N / 2) * pitch for i in range(N)]

top_ports = [pp.Port("top_{}".format(i), (xs_top[i], 0), 0.5, 270) for i in range(N)]

bottom_ports = [
    pp.Port("bottom_{}".format(i), (xs_bottom[i], -100), 0.5, 90) for i in range(N)
]

c = pp.Component(name="connect_bundle")

for p1, p2 in zip(top_ports, bottom_ports):
    route = pp.routing.get_route(p1, p2)
    c.add(route.references)

c

**solution**: river routing routes several ports without collisions

In [ ]:
c = pp.Component(name="connect_bundle")
routes = pp.routing.get_bundle(top_ports, bottom_ports)
for route in routes:
    c.add(route.references)
    
c

In [ ]:
import pp
ys_right = [0, 10, 20, 40, 50, 80]
pitch = 127.0
N = len(ys_right)
ys_left = [(i - N / 2) * pitch for i in range(N)]

right_ports = [pp.Port(f"R_{i}", (0, ys_right[i]), 0.5, 180) for i in range(N)]
left_ports = [pp.Port(f"L_{i}".format(i), (-400, ys_left[i]), 0.5, 0) for i in range(N)]

# you can also mess up the port order and it will sort them by default
left_ports.reverse()

c = pp.Component(name="connect_bundle2")
routes = pp.routing.get_bundle(right_ports, left_ports, sort_ports=True)
for route in routes:
    c.add(route.references)
c

In [ ]:
xs_top = [0, 10, 20, 40, 50, 80]
pitch = 127.0
N = len(xs_top)
xs_bottom = [(i - N / 2) * pitch for i in range(N)]

top_ports = [pp.Port("top_{}".format(i), (xs_top[i], 0), 0.5, 270) for i in range(N)]

bottom_ports = [
    pp.Port("bottom_{}".format(i), (xs_bottom[i], -400), 0.5, 90) for i in range(N)
]

c = pp.Component(name="connect_bundle")
routes = pp.routing.get_bundle(top_ports, bottom_ports)
for route in routes:
    c.add(route.references)

c

## Routing bundles with waypoints

In [ ]:
import numpy as np
import pp


@pp.cell
def test_connect_bundle_waypoints():
    """Connect bundle of ports with bundle of routes following a list of waypoints."""
    xs1 = np.arange(10) * 5 - 500.0
    N = xs1.size
    ys2 = np.array([0, 5, 10, 20, 25, 30, 40, 55, 60, 75]) + 500.0

    ports1 = [pp.Port(f"A_{i}", (xs1[i], 0), 0.5, 90) for i in range(N)]
    ports2 = [pp.Port(f"B_{i}", (0, ys2[i]), 0.5, 180) for i in range(N)]

    c = pp.Component()
    way_points = [
        ports1[0].position,
        ports1[0].position + (0, 100),
        ports1[0].position + (200, 100),
        ports1[0].position + (200, -200),
        ports1[0].position + (0, -200),
        ports1[0].position + (0, -350),
        ports1[0].position + (400, -350),
        (ports1[0].x + 400, ports2[-1].y),
        ports2[-1].position,
    ]

    routes = pp.routing.get_bundle_from_waypoints(ports1, ports2, way_points)
    for route in routes:
        c.add(route.references)

    return c

cell = test_connect_bundle_waypoints()
cell

## Route bundles through corners

In [ ]:
import pp
from pp.cell import cell
from pp.component import Component
from pp.port import Port

@cell
def test_connect_corner(N=6, config="A"):

    d = 10.0
    sep = 5.0
    top_cell = pp.Component(name="connect_corner")

    if config in ["A", "B"]:
        a = 100.0
        ports_A_TR = [
            Port("A_TR_{}".format(i), (d, a / 2 + i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_TL = [
            Port("A_TL_{}".format(i), (-d, a / 2 + i * sep), 0.5, 180) for i in range(N)
        ]
        ports_A_BR = [
            Port("A_BR_{}".format(i), (d, -a / 2 - i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_BL = [
            Port("A_BL_{}".format(i), (-d, -a / 2 - i * sep), 0.5, 180)
            for i in range(N)
        ]

        ports_A = [ports_A_TR, ports_A_TL, ports_A_BR, ports_A_BL]

        ports_B_TR = [
            Port("B_TR_{}".format(i), (a / 2 + i * sep, d), 0.5, 90) for i in range(N)
        ]
        ports_B_TL = [
            Port("B_TL_{}".format(i), (-a / 2 - i * sep, d), 0.5, 90) for i in range(N)
        ]
        ports_B_BR = [
            Port("B_BR_{}".format(i), (a / 2 + i * sep, -d), 0.5, 270) for i in range(N)
        ]
        ports_B_BL = [
            Port("B_BL_{}".format(i), (-a / 2 - i * sep, -d), 0.5, 270)
            for i in range(N)
        ]

        ports_B = [ports_B_TR, ports_B_TL, ports_B_BR, ports_B_BL]

    elif config in ["C", "D"]:
        a = N * sep + 2 * d
        ports_A_TR = [
            Port("A_TR_{}".format(i), (a, d + i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_TL = [
            Port("A_TL_{}".format(i), (-a, d + i * sep), 0.5, 180) for i in range(N)
        ]
        ports_A_BR = [
            Port("A_BR_{}".format(i), (a, -d - i * sep), 0.5, 0) for i in range(N)
        ]
        ports_A_BL = [
            Port("A_BL_{}".format(i), (-a, -d - i * sep), 0.5, 180) for i in range(N)
        ]

        ports_A = [ports_A_TR, ports_A_TL, ports_A_BR, ports_A_BL]

        ports_B_TR = [
            Port("B_TR_{}".format(i), (d + i * sep, a), 0.5, 90) for i in range(N)
        ]
        ports_B_TL = [
            Port("B_TL_{}".format(i), (-d - i * sep, a), 0.5, 90) for i in range(N)
        ]
        ports_B_BR = [
            Port("B_BR_{}".format(i), (d + i * sep, -a), 0.5, 270) for i in range(N)
        ]
        ports_B_BL = [
            Port("B_BL_{}".format(i), (-d - i * sep, -a), 0.5, 270) for i in range(N)
        ]

        ports_B = [ports_B_TR, ports_B_TL, ports_B_BR, ports_B_BL]

    if config in ["A", "C"]:
        for ports1, ports2 in zip(ports_A, ports_B):
            routes = pp.routing.get_bundle(ports1, ports2, waveguide='nitride', radius=5)
            for route in routes:
                top_cell.add(route.references)

    elif config in ["B", "D"]:
        for ports1, ports2 in zip(ports_A, ports_B):
            routes = pp.routing.get_bundle(ports2, ports1, waveguide='nitride', radius=5)
            for route in routes:
                top_cell.add(route.references)

    return top_cell


c = test_connect_corner(config='A')
c

In [ ]:
c = test_connect_corner(config='C')
c

In [ ]:
@cell
def test_connect_bundle_udirect(dy=200, angle=270):

    xs1 = [-100, -90, -80, -55, -35, 24, 0] + [200, 210, 240]

    axis = "X" if angle in [0, 180] else "Y"

    pitch = 10.0
    N = len(xs1)
    xs2 = [70 + i * pitch for i in range(N)]

    if axis == "X":
        ports1 = [Port(f"top_{i}", (0, xs1[i]), 0.5, angle) for i in range(N)]

        ports2 = [
            Port(f"bottom_{i}", (dy, xs2[i]), 0.5, angle) for i in range(N)
        ]

    else:
        ports1 = [Port(f"top_{i}", (xs1[i], 0), 0.5, angle) for i in range(N)]

        ports2 = [
            Port(f"bottom_{i}", (xs2[i], dy), 0.5, angle) for i in range(N)
        ]

    top_cell = Component(name="connect_bundle_udirect")
    routes = pp.routing.get_bundle(ports1, ports2, waveguide='nitride', radius=10.)
    for route in routes:
        top_cell.add(route.references)

    return top_cell


c = test_connect_bundle_udirect()
c

In [ ]:
@cell
def test_connect_bundle_u_indirect(dy=-200, angle=180):
    xs1 = [-100, -90, -80, -55, -35] + [200, 210, 240]
    axis = "X" if angle in [0, 180] else "Y"
    pitch = 10.0
    N = len(xs1)
    xs2 = [50 + i * pitch for i in range(N)]

    a1 = angle
    a2 = a1 + 180

    if axis == "X":
        ports1 = [Port("top_{}".format(i), (0, xs1[i]), 0.5, a1) for i in range(N)]

        ports2 = [Port("bottom_{}".format(i), (dy, xs2[i]), 0.5, a2) for i in range(N)]

    else:
        ports1 = [Port("top_{}".format(i), (xs1[i], 0), 0.5, a1) for i in range(N)]

        ports2 = [Port("bottom_{}".format(i), (xs2[i], dy), 0.5, a2) for i in range(N)]

    top_cell = Component("connect_bundle_u_indirect")
    routes = pp.routing.get_bundle(ports1, ports2, bend_factory=pp.components.bend_euler, waveguide='nitride', radius=10)
    for route in routes:
        top_cell.add(route.references)

    return top_cell


c = test_connect_bundle_u_indirect(angle=0)
c

In [ ]:
import pp

@pp.cell
def test_north_to_south():
    dy = 200.0
    xs1 = [-500, -300, -100, -90, -80, -55, -35, 200, 210, 240, 500, 650]

    pitch = 10.0
    N = len(xs1)
    xs2 = [-20 + i * pitch for i in range(N // 2)]
    xs2 += [400 + i * pitch for i in range(N // 2)]

    a1 = 90
    a2 = a1 + 180

    ports1 = [pp.Port("top_{}".format(i), (xs1[i], 0), 0.5, a1) for i in range(N)]
    ports2 = [pp.Port("bottom_{}".format(i), (xs2[i], dy), 0.5, a2) for i in range(N)]

    c = pp.Component()
    routes = pp.routing.get_bundle(ports1, ports2, waveguide='nitride', auto_widen=False)
    for route in routes:
        c.add(route.references)

    return c


c = test_north_to_south()
c

In [ ]:
def demo_connect_bundle():
    """ combines all the connect_bundle tests """
    y = 400.0
    x = 500
    y0 = 900
    dy = 200.0
    c = Component("connect_bundle")
    for j, s in enumerate([-1, 1]):
        for i, angle in enumerate([0, 90, 180, 270]):
            _cmp = test_connect_bundle_u_indirect(dy=s * dy, angle=angle)
            _cmp_ref = _cmp.ref(position=(i * x, j * y))
            c.add(_cmp_ref)

            _cmp = test_connect_bundle_udirect(dy=s * dy, angle=angle)
            _cmp_ref = _cmp.ref(position=(i * x, j * y + y0))
            c.add(_cmp_ref)

    for i, config in enumerate(["A", "B", "C", "D"]):
        _cmp = test_connect_corner(config=config)
        _cmp_ref = _cmp.ref(position=(i * x, 1700))
        c.add(_cmp_ref)

    #_cmp = test_facing_ports()
    #_cmp_ref = _cmp.ref(position=(800, 1820))
    #c.add(_cmp_ref)

    return c

c = demo_connect_bundle()
c

In [ ]:
import pp

c = pp.Component('route_bend_5um')
c1 = c <<pp.components.mmi2x2()
c2 = c <<pp.components.mmi2x2()
c2.move((100, 50))
routes = pp.routing.get_bundle([c1.ports["E0"], c1.ports['E1']], [c2.ports["W0"], c2.ports['W1']], radius=5)
for route in routes:
    c.add(route.references)
c

In [ ]:
import pp

c = pp.Component('route_bend_20um')
c1 = c <<pp.components.mmi2x2()
c2 = c <<pp.components.mmi2x2()
c2.move((80, 40))
routes = pp.routing.get_bundle([c1.ports["E0"], c1.ports['E1']], [c2.ports["W0"], c2.ports['W1']], radius=5)
for route in routes:
    c.add(route.references)
c

In [ ]:
import pp

c = pp.Component('electrical')
c1 = c <<pp.components.pad()
c2 = c <<pp.components.pad()
c2.move((200, 100))
routes = pp.routing.get_bundle([c1.ports["E"]], [c2.ports["W"]], route_filter=pp.routing.get_route_from_waypoints_electrical, waveguide='metal_routing')
for route in routes:
    c.add(route.references)
c

In [ ]:
c = pp.Component("routes_sample")
pad_array = pp.components.pad_array(pitch=150, port_list=("S",))
c1 = c << pad_array
c2 = c << pad_array
c2.rotate(90)
c2.movex(1000)
c2.ymax = -200

routes_bend180 = pp.routing.get_routes_bend180(
    ports=c2.get_ports_list(), radius=75 / 2, waveguide="metal_routing"
)
c.add(routes_bend180.references)

routes = pp.routing.get_bundle(
    c1.get_ports_list(), routes_bend180.ports, waveguide="metal_routing"
)
for route in routes:
    c.add(route.references)
c

In [ ]:
# Problem: sometimes 90 degrees routes do not have enough space for a Manhattan route

In [ ]:
import pp

c = pp.Component('route_fail_1')
c1 = c <<pp.components.nxn(east=3, ysize=20)
c2 = c <<pp.components.nxn(west=3)
c2.move((80, 0))
routes = pp.routing.get_bundle(c1.get_ports_list(prefix='E'), c2.get_ports_list(prefix='W'), auto_widen=False)
for route in routes:
    c.add(route.references)
c

In [ ]:
c = pp.Component("route_fail_2")
pitch = 2.0
ys_left = [0, 10, 20]
N = len(ys_left)
ys_right = [(i - N / 2) * pitch for i in range(N)]

right_ports = [pp.Port(f"R_{i}", (0, ys_right[i]), 0.5, 180) for i in range(N)]
left_ports = [pp.Port(f"L_{i}", (-50, ys_left[i]), 0.5, 0) for i in range(N)]
left_ports.reverse()
routes = pp.routing.get_bundle(right_ports, left_ports, radius=5)

for i, route in enumerate(routes):
    c.add(route.references)
c

In [ ]:
# Solution: Add Sbend routes
import pp

c = pp.Component('route_solution_1_get_bundle_sbend')
c1 = c <<pp.components.nxn(east=3, ysize=20)
c2 = c <<pp.components.nxn(west=3)
c2.move((80, 0))
routes = pp.routing.get_bundle_sbend(c1.get_ports_list(prefix='E'), c2.get_ports_list(prefix='W'))
c.add(routes.references)
c

In [ ]:
routes

In [ ]:
c = pp.Component('route_solution_2_get_bundle_sbend')
route = pp.routing.get_bundle_sbend(right_ports, left_ports)
c.add(route.references)

In [ ]:
import pp

c = pp.Component('route_solution_1_get_routes')
c1 = c <<pp.components.nxn(east=3, ysize=20)
c2 = c <<pp.components.nxn(west=3)
c2.move((80, 0))
routes = pp.routing.get_routes(c1.get_ports_list(prefix='E'), c2.get_ports_list(prefix='W'))
c.add(routes.references)
c

In [ ]:
c = pp.Component('route_solution_2_get_routes')
routes = pp.routing.get_routes(right_ports, left_ports)
c.add(routes.references)

## Route electrical ports

In [ ]:
import pp

mzi = pp.components.mzi2x2(with_elec_connections=True)
mzi

In [ ]:
pp.components.pad_array?

In [ ]:
import pp

c = pp.Component("mzi_with_pads_sample")
mzi = pp.components.mzi2x2(with_elec_connections=True)
pads = pp.components.pad_array(n=3, port_list=["S"])
p = c << pads
mzir = c << mzi
p.move((-150, 250))

routes = pp.routing.get_bundle(
    p.ports,
    mzir.get_ports_list(port_type='dc'),
    waveguide='metal_routing'
)
for route in routes:
    c.add(route.references)
c

In [ ]:
import pp

c = pp.components.mzi2x2(with_elec_connections=True)
cc = pp.routing.add_electrical_pads_top(component=c)
cc

In [ ]:
import pp

c = pp.components.straight_with_heater()
cc = pp.routing.add_electrical_pads_top(component=c)
cc

In [ ]:
import pp

c = pp.components.mzi2x2(with_elec_connections=True)
cc = pp.routing.add_electrical_pads_shortest(component=c)
cc

In [ ]:
import pp

c = pp.components.mzi2x2(with_elec_connections=True)
cc = pp.routing.add_electrical_pads(component=c, fanout_length=100)
cc

In [ ]:
import pp

c = pp.components.cross(length=100, layer=pp.LAYER.M3, port_type="dc")
c.move((20, 50))
cc = pp.routing.add_electrical_pads_shortest(component=c)
cc

## Routing to grating couplers

### Route to Fiber Array

Routing allows you to define routes to optical or electrical IO (grating couplers or electrical pads)

In [ ]:
import numpy as np
import pp
from pp import LAYER
from pp import Port


@pp.cell
def big_device(w=400.0, h=400.0, N=16, port_pitch=15.0, layer=LAYER.WG, wg_width=0.5):
    """ big component with N ports on each side """
    component = pp.Component()
    p0 = np.array((0, 0))
    dx = w / 2
    dy = h / 2

    points = [[dx, dy], [dx, -dy], [-dx, -dy], [-dx, dy]]
    component.add_polygon(points, layer=layer)
    port_params = {"layer": layer, "width": wg_width}
    for i in range(N):
        port = Port(
            name="W{}".format(i),
            midpoint=p0 + (-dx, (i - N / 2) * port_pitch),
            orientation=180,
            **port_params
        )
        component.add_port(port)

    for i in range(N):
        port = Port(
            name="E{}".format(i),
            midpoint=p0 + (dx, (i - N / 2) * port_pitch),
            orientation=0,
            **port_params
        )
        component.add_port(port)

    for i in range(N):
        port = Port(
            name="N{}".format(i),
            midpoint=p0 + ((i - N / 2) * port_pitch, dy),
            orientation=90,
            **port_params
        )
        component.add_port(port)

    for i in range(N):
        port = Port(
            name="S{}".format(i),
            midpoint=p0 + ((i - N / 2) * port_pitch, -dy),
            orientation=-90,
            **port_params
        )
        component.add_port(port)
    return component


component = big_device(N=10)
c = pp.routing.add_fiber_array(component=component, radius=10., fanout_length=60.0)
c

In [ ]:
import pp

c = pp.components.ring_double(width=0.8)
cc = pp.routing.add_fiber_array(component=c, taper_length=150)
cc

In [ ]:
cc.get_settings()

In [ ]:
cc.pprint()

You can also mix and match `TE` and `TM` grating couplers

In [ ]:
c = pp.components.mzi2x2()
gcte = pp.components.grating_coupler_te
gctm = pp.components.grating_coupler_tm

cc = pp.routing.add_fiber_array(
    component=c,
    optical_routing_type=2,
    grating_coupler=[gctm, gcte, gctm, gcte],
    radius=20
)
cc

### Route to fiber single

In [ ]:
import pp

c = pp.components.ring_single()
cc = pp.routing.add_fiber_single(component=c)
cc

In [ ]:
import pp

c = pp.components.ring_single()
cc = pp.routing.add_fiber_single(component=c, with_align_ports=False)
cc

In [ ]:
c = pp.components.mmi2x2()
cc = pp.routing.add_fiber_single(component=c, with_align_ports=False)
cc

In [ ]:
c = pp.components.mmi1x2()
cc = pp.routing.add_fiber_single(component=c, with_align_ports=False, fiber_spacing=150)
cc

In [ ]:
c = pp.components.mmi1x2()
cc = pp.routing.add_fiber_single(component=c, with_align_ports=False, fiber_spacing=50)
cc

In [ ]:
c = pp.components.crossing()
cc = pp.routing.add_fiber_single(component=c, with_align_ports=False)
cc

In [ ]:
c = pp.components.cross(length=200, width=2)
cc = pp.routing.add_fiber_single(component=c, with_align_ports=False)
cc

## Routing with custom waypoins

Sometimes you need to set up a route with custom waypoints

In [ ]:
import pp

c = pp.Component('waypoints_sample')

w = pp.components.straight()
left = c << w
right = c << w
right.move((100, 80))

obstacle = pp.components.rectangle(size=(100, 10))
obstacle1 = c << obstacle
obstacle2 = c << obstacle
obstacle1.ymin=40
obstacle2.xmin=25


p0x, p0y = left.ports['E0'].midpoint
p1x, p1y = right.ports['E0'].midpoint
o = 10 # vertical offset to overcome bottom obstacle
ytop = 20


routes = pp.routing.get_route_from_waypoints(
    [
        (p0x, p0y),
        (p0x + o, p0y),
        (p0x + o, ytop),
        (p1x + o, ytop),
        (p1x + o, p1y),
        (p1x, p1y),
    ],
)
c.add(routes.references)
      
c

## Routing with path length matching

Sometimes you need to set up a route a bundle of ports that need to keep the same lengths

In [ ]:
import pp

c = pp.Component('path_length_match_sample')
dy = 2000.0
xs1 = [-500, -300, -100, -90, -80, -55, -35, 200, 210, 240, 500, 650]

pitch = 100.0
N = len(xs1)
xs2 = [-20 + i * pitch for i in range(N)]

a1 = 90
a2 = a1 + 180

ports1 = [pp.Port(f"top_{i}", (xs1[i], 0), 0.5, a1) for i in range(N)]
ports2 = [pp.Port(f"bottom_{i}", (xs2[i], dy), 0.5, a2) for i in range(N)]

routes = pp.routing.get_bundle_path_length_match(ports1, ports2)

for route in routes:
    c.add(route.references)
    print(route.length)
c

### Add extra length

You can also add some extra length to all the routes

In [ ]:
import pp
c = pp.Component("path_length_match_sample")

dy = 2000.0
xs1 = [-500, -300, -100, -90, -80, -55, -35, 200, 210, 240, 500, 650]

pitch = 100.0
N = len(xs1)
xs2 = [-20 + i * pitch for i in range(N)]

a1 = 90
a2 = a1 + 180

ports1 = [pp.Port(f"top_{i}", (xs1[i], 0), 0.5, a1) for i in range(N)]
ports2 = [pp.Port(f"bottom_{i}", (xs2[i], dy), 0.5, a2) for i in range(N)]

routes = pp.routing.get_bundle_path_length_match(
    ports1, ports2, extra_length=44, waveguide='nitride'
)
for route in routes:
    c.add(route.references)
    print(route.length)
c

### increase number of loops

You can also increase the number of loops

In [ ]:
c = pp.Component("path_length_match_sample")

dy = 2000.0
xs1 = [-500, -300, -100, -90, -80, -55, -35, 200, 210, 240, 500, 650]

pitch = 200.0
N = len(xs1)
xs2 = [-20 + i * pitch for i in range(N)]

a1 = 90
a2 = a1 + 180

ports1 = [pp.Port(f"top_{i}", (xs1[i], 0), 0.5, a1) for i in range(N)]
ports2 = [pp.Port(f"bottom_{i}", (xs2[i], dy), 0.5, a2) for i in range(N)]

routes = pp.routing.get_bundle_path_length_match(ports1, ports2, nb_loops=2, auto_widen=False)
for route in routes:
    c.add(route.references)
    print(route.length)
c